
## Live Betting-Webscrapping

Medium-Link: https://medium.datadriveninvestor.com/make-money-with-python-the-sports-arbitrage-project-3b09d81a0098


**Importing libraries**

We need to import Optionsto change chromedriver’s default options; Select for selecting within dropdown menus andBy, WebDriverWait, EC,timeto wait for a certain condition to occur.

In [ ]:
#https://medium.com/@shivangisareen/for-anyone-using-jupyter-notebook-installing-packages-18a9468d0c1c

In [1]:
#import libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# import libraries
import subprocess
import pandas as pd
import pickle

In [2]:
# evtl schlechte Lösung um FuzzyWuzzy und Sympy zu importiren
# -> nicth sicher, ob ich mit sys.path mehr kaputt mache bzw Fuzzywuzzy nachher überhaupt anwendbar ist.

import sys
tmppath = sys.path.copy()
sys.path = ['/Users/marcotilli/Desktop/Projects/Betscraping', '/Users/marcotilli/opt/miniconda3/envs/betscrap/lib/python38.zip', '/Users/marcotilli/opt/miniconda3/envs/betscrap/lib/python3.8', '/Users/marcotilli/opt/miniconda3/envs/betscrap/lib/python3.8/lib-dynload', '/Users/marcotilli/opt/miniconda3/envs/betscrap/lib/python3.8/site-packages']

from fuzzywuzzy import process, fuzz
from sympy import symbols, Eq, solve

sys.path = tmppath

/Users/marcotilli/opt/miniconda3/envs/betscrap/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [39]:
import Scrappers
from tipico import scrap_tipico

In [47]:
df_tipico = scrap_tipico()

TypeError: scrap_tipico() missing 1 required positional argument: 'sport'

**Changing Chromedriver default options**

To keep scraping while doing something else on the computer, we need to use the headless mode. To do so, we write the following code:

In [10]:
#changing chromedriver default options
options = Options()
options.headless = False
#options.add_argument('window-size=2400x1080') 

web = 'https://sports.tipico.de/en/live/' 
#execute chromedriver with edited options
driver = webdriver.Chrome('./chromedriver', options=options)
driver.get(web)
driver.maximize_window()

# accept Cookies
accept = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="_evidon-accept-button"]')))
accept.click()

Breaking down the code:
- *options = Options()* creates an instance of the Options class
- *options.headless = True* turns on the headless mode
- *options.add_argument(‘window-size=1920x1080’)* opens the windows in a customized size on the background
- *webdriver.Chrome(path, options = options)* applies the changes we made in the chromedriver
- *driver.get* opens the browser
- *web* represents the betting site’s URL, while path represents the chromedriver’s path in your computer

If you don’t want to work in headless mode, then write option.headless=False and use driver.maximize_window() instead of options.add_argument(‘window-size=1920x1080’) as shown in the full code.

- *Select('...')* selects dropdown menus
- *first_dropdown* one of the three dropdowns that contain betting markets
- *first_dropdown.select_by_visible_text()* selects an element inside the dropdown menu with the help of the betting market’s name.

In [11]:
#Initialize your storage
teams = []
x12 = []
over_under = []
btts = []
odds_events = []

#select values from dropdown
first_dropdown = Select(WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/main/main/section/div/div/div[4]/div[1]/div/div[1]/select[1]'))))
second_dropdown = Select(WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/main/main/section/div/div/div[4]/div[1]/div/div[1]/select[2]'))))
third_dropdown = Select(WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/main/main/section/div/div/div[4]/div[1]/div/div[1]/select[3]'))))
first_dropdown.select_by_visible_text('3-Way')
second_dropdown.select_by_visible_text('Over/Under')
third_dropdown.select_by_visible_text('Both teams to score?')

**Looking for ‘live events’ and ‘sports titles’**

Before getting the data, we need first to look for live events. Also, to simplify the analysis, we’ll choose only ‘football’ in sports’ names. To do so, we write the following code:

- *box* represents the box that contains sports events. The site also contains upcoming events, which we don’t need for this analysis.
- *driver.find_element_by_xpath()* helps us find an element within the website through the ‘Xpath’ of that element.

Unlike Xpaths from most elements we worked with before, the ‘live events’ element is trickier to get. For that reason, we need to be very specific. Then we use the contains option to match an element that contains Program_LIVE inside thetestid within a div tag. Check the picture below to find Program_LIVE

In [12]:
#Looking for live events 'Program_LIVE'
box = driver.find_element_by_xpath('//div[contains(@testid, "Program_LIVE")]')
#Looking for 'sports titles'
sport_title = box.find_elements_by_class_name('SportTitle-styles-sport')

**1) Find ‘empty events’**

Unlike pre-match games, odds in live games change quickly and could be suspended for a couple of seconds at any time. When betting odds are momentarily suspended, the odds are either locked or empty. Locked odds are easy to scrape, but empty odds are harder. That’s why we need to locate those empty odds

- *parent* represents the ‘parent node’ of the *sport* element, while *grandparent* is its ‘grandparent node’ that represents the whole ‘football’ container inside live events

**2) Remove empty_events from single_row_events**

We find single row events and remove all empty events inside

**3) Getting live odds**

Each dropdown selected before will give odds for 3 markets that are extracted

- *for match in single_row_events* loops through all the matches inside the ‘single_row_events’ list
- *odd_events* represent each event with odds available
- *match.find_elements_by_class_name(‘EventOddGroup-styles-odd-groups’)* helps us find all the ‘odds_event’ within every match. To find the class name *‘EventOddGroup-styles-odd-groups’* just right-click and inspect the code behind ‘odds box’ as we did before for empty odds
- *for team in match.find_elements_by_class_name(‘EventTeams-styles-titles’)* loops through the elements with the class name 'EventTeams-styles-titles' within the ‘match’ node. Matches have 2 teams (home and away team); that’s why we need to loop through them and obtain their name with *team.text*. Then they’re stored with the append-method in the *teams* list
- *for odd_event in odds_events* loops over the total number of live matches on the betting site.
- *for n, box in enumerate(odds_events)* loops through all ‘odds boxes’ inside a match. In the beginning, we changed the dropdowns to ‘3-way,’ ‘Over/Under’ and ‘Both Teams to Score,’ so they’re the odds we’ll scrape
- *rows = box.find_elements_by_xpath('.//*')* gives all the children nodes (odds) inside the box element.
- *n==0* means ‘only take values from the first box.’ In this case, the first box is the ‘3-way’ box and is stored in the *x12* list
- *rows[0]* tells Python ‘only pick the first row on each odds box.’ With this, we ignored the second row in double-row events.

In [13]:
for sport in sport_title:
    # selecting only football
    if sport.text == 'Football':
        parent = sport.find_element_by_xpath('./..') #immediate parent node
        grandparent = parent.find_element_by_xpath('./..') #grandparent node = the whole 'football' container
        
        #1 #empty groups
        try:
            empty_groups = grandparent.find_elements_by_class_name('EventOddGroup-styles-empty-group')
            empty_events = [empty_group.find_element_by_xpath('./..') for empty_group in empty_groups]####
        except:
            pass
        
        #Single row event
        single_row_events = grandparent.find_elements_by_class_name('EventRow-styles-event-row')
        #2 #remove empty events from single_row_events
        try:
            #empty_events
            single_row_events = [single_row_event for single_row_event in single_row_events if single_row_event not in empty_events]
        except:
            pass
        
        #3 #Getting Data
        for match in single_row_events:
            odds_event = match.find_elements_by_class_name('EventOddGroup-styles-odd-groups')
            odds_events.append(odds_event)
            # teams
            for team in match.find_elements_by_class_name('EventTeams-styles-titles'):
                teams.append(team.text)
        for odds_event in odds_events:
            for n, box in enumerate(odds_event):
                rows = box.find_elements_by_xpath('.//*')
                if n == 0:
                    x12.append(rows[0].text)
                # over/under + goal line
                if n == 1:
                    parent = box.find_element_by_xpath('./..')
                    goals = parent.find_element_by_class_name('EventOddGroup-styles-fixed-param-text').text
                    over_under.append(goals+'\n'+rows[0].text)
                # both teams to score
                if n == 2:
                    btts.append(rows[0].text)



In [14]:
driver.quit()

### Make it Readable

Now we have to make the data readable and do some pre-cleaning with Pandas. Then we’ll save the data with Pickle.

- *dict_gambling* is the dictionary that stores all the lists that contain the odds scraped
- *pd.DataFrame.from_dict(dict_gambling)* turn the dictionary into a dataframe df_tipico,so we can read it and work with it easily
- *df_tipico.applymap(lambda x: x.strip() if isinstance(x, str) else x)* cleans all the leading and trailing white spaces that the odds might have with thestrip method.
- *open('...', 'wb')* opens a file named *df_tipico* in the mode *‘write bytes’* (wb). We save this in the variable *output*
- *pickle.dump(df_tipico, output)* saves the dataframe created in the file named df_tipico. We used the same name for both the file and the dataframe, but they could be named differently
- *output.close()* closes the file

In [15]:
#7 #unlimited columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#Storing lists within dictionary
dict_gambling = {'Teams': teams, 'btts': btts,
                 'Over/Under': over_under, '3-way': x12}
#Presenting data in dataframe
df_tipico = pd.DataFrame.from_dict(dict_gambling)
#clean leading and trailing whitespaces of all odds
df_tipico = df_tipico.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#Save data with Pickle
output = open('df_tipico', 'wb')
pickle.dump(df_tipico, output)
output.close()
print(df_tipico)

                                            Teams         btts       Over/Under               3-way
0                     UD Las Palmas Atl.\nCadiz B   3.80\n1.20  0.5\n1.45\n2.55    2.80\n2.10\n3.80
1          UD San Sebastian\nCD Atletico Baleares   2.10\n1.65  1.5\n1.40\n2.70    1.35\n4.10\n9.00
2                  Rayo Majadahonda\nLas Rozas CF   3.80\n1.20  0.5\n1.40\n2.70    2.10\n2.30\n5.50
3                  UC Sampdoria\nAtalanta Bergamo   2.00\n1.70  2.5\n2.20\n1.60    4.50\n3.30\n1.80
4                      Calcio Lecco\nJuventus U23   1.70\n2.00  3.5\n1.70\n2.00  1.06\n10.00\n20.00
5                       Carpi FC 1909\nFermana FC   2.80\n1.35  1.5\n2.00\n1.70    2.40\n2.60\n3.40
6                     Sassuolo U19\nCFC Genoa U19   1.37\n2.70  3.5\n2.10\n1.60    1.55\n4.00\n5.00
7                Keciörengücü\nYilport Samsunspor   2.40\n1.50  1.5\n1.75\n1.95   1.35\n4.00\n13.00
8                Sk Rapid Vienna II\nFC Liefering   2.40\n1.50  1.5\n1.70\n2.10   1.30\n4.00\n15.00
